# Introduction

### Problem Description

Example: alloy of two metals, two control variables: proportion of one metal to the other and intensity of heat treatment, loss is fragility of alloy.

### Software Requirements

This notebook does not use any packages beyond a list of those that are quite popular in scientific computing. Use conda or pip to install any of them.

# General Preparations

# Template of Experiments

TODO: for each capacity of xgboost and number of observations compute average cross-validation error and average score in estimated minimum over multiple runs as well as KDE of estimated minimums.

TODO: Pass design of experiment as a string argument that is matched to a function.

TODO: Two benchmarks - random guessing over all square and taking a minimum point over training sample (note that there is noise in measured targets).

# Experiments

### Himmelblau's Function and Random DoE

Himmelblau's function is: $$f(x, y) = (x^2+y-11)^2 + (x+y^2-7)^2.$$

# Conclusion

To be continued.